In [ ]:
# install pydrake

# try:
#   import pydrake
#   import underactuated
# except ImportError:
#   !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
#   from jupyter_setup import setup_underactuated
#   setup_underactuated()


In [ ]:
from pydrake.all import eq, MathematicalProgram, Solve, Variable

import matplotlib.pyplot as plt
import numpy as np
import importlib

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator

from pydrake.all import (
    DiagramBuilder, Simulator
)

from pydrake.multibody.tree import (
    JointActuatorIndex
)

from pydrake.geometry import SceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.trajectories import PiecewisePolynomial
from pydrake.solvers.snopt import SnoptSolver

In [ ]:
# set up constants

N = 7      # number of collocation points
n_x = 6
n_u = 3

# where we start, where we go, mass?, 
# start position
alpha_0 = -23.45   #longitude degrees
beta_0 = -2.94     #latitude degrees
r_0 = 3474.8       #radius of the moon, in km
phi_0 = 90         #pitch degrees
psi_0 = 0          #yaw degrees

# final position
alpha_N = -23.45   #degrees
beta_N = -1.94     #degrees
r_N = 3474.8       #radius of the moon, in km
phi_N = 90
psi_N = 0

# initial control input 
T_0 = 0
phiddot_N = 0
psiddot_N = 0

# final control input
T_N = 0
phiddot_N = 0
psiddot_N = 0

m = 8321.09        #kg
g = 3.711          #m/s^2


In [ ]:
prog = MathematicalProgram()
x = np.zeros((N,n_x),dtype="object")
u = np.zeros((N,n_u),dtype="object")
t = np.zeros((N,1),dtype="object")

for i in range(N):
    x[i] = prog.NewContinuousVariables(n_x,"x_" + str(i))
    u[i] = prog.NewContinuousVariabels(n_u,"u_" + str(i))
#     t[i] = prog.NewContinuousVariables(1,"t_" + str(i))



In [ ]:
# TODO: Add cost (sum of square of all inputs)

In [ ]:
# TODO: Add dynamic constraint

In [ ]:
# TODO: Add kinematic constraint

In [ ]:
# TODO: Add input/state limitation as bounding box constraints

In [ ]:
# TODO: Set up solver initial guess

In [ ]:
# Set up solver

solver = SnoptSolver()
result = solver.Solve(prog)
  
x_sol = result.GetSolution(x)
u_sol = result.GetSolution(u)
t_sol = result.GetSolution(t)

# TODO: generate trajectory based on the solutions